In [1]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fitsio
import desimodel.io
import desitarget.mtl
import desisim.quickcat
from astropy.io import fits
from astropy.table import Table, Column, vstack, hstack
import json
import shutil
import myquickcat
from importlib import reload

from desitarget.targetmask import desi_mask, obsconditions
from collections import Counter
import subprocess

%matplotlib inline

In [16]:
full_targets = Table.read("targets/subset_dr8_mtl_dark_gray_NGC.fits")
zcat = Table.read("strategy_A/zcat/dark2_dark3_zcat.fits")
full_mtl = Table.read("strategy_A/targets/dark2_dark3_subset_dr8_mtl_dark_gray_northern_cap.fits")
truth = Table.read("targets/subset_truth_dr8_mtl_dark_gray_NGC.fits")

ii = np.in1d(full_mtl['TARGETID'], zcat['TARGETID'])
in_mtl = full_mtl[ii]

ii = np.in1d(full_targets['TARGETID'], zcat['TARGETID'])
in_targets = full_targets[ii]

print(len(zcat), len(in_mtl), len(in_targets))
in_mtl.sort(keys='TARGETID')
zcat.sort(keys='TARGETID')
in_targets.sort(keys='TARGETID')
truth.sort(keys='TARGETID')


1024622 1024622 1024622


In [3]:
assert np.all(in_mtl['TARGETID']==zcat['TARGETID'])
assert np.all(in_mtl['TARGETID']==in_targets['TARGETID'])

In [38]:
zcat_large = zcat[(zcat['NUMOBS']>1) & (zcat['SPECTYPE']=='QSO') & (zcat['Z']<2.15)]
in_mtl_large = in_mtl[(zcat['NUMOBS']>1) & (zcat['SPECTYPE']=='QSO') & (zcat['Z']<2.15)]
in_targets_large = in_targets[(zcat['NUMOBS']>1) & (zcat['SPECTYPE']=='QSO') & (zcat['Z']<2.15)]
print(len(zcat_large), 100*len(zcat_large)/len(zcat))

ii = (zcat['NUMOBS']>3) & (zcat['Z']<2.15) & (zcat['SPECTYPE']=='QSO')
zcat_large = zcat[ii]
in_mtl_large = in_mtl[ii]
in_targets_large = in_targets[ii]
print(len(zcat_large), 100*len(zcat_large)/len(zcat))

8441 0.8238160023891737
9 0.0008783727072032418


In [39]:
names = {}
name_id = {}
for i in range(len(in_mtl_large)):
    a = desi_mask.names(in_mtl_large['DESI_TARGET'][i])
    n = ''
    for aa in a:
        n = n +' '+ aa
    if n not in names.keys():
        names[n] = 1
    else:
        names[n] += 1
    name_id[in_mtl_large['TARGETID'][i]] = n


In [40]:
names

{' QSO QSO_SOUTH STD_FAINT BGS_ANY MWS_ANY': 1,
 ' QSO QSO_SOUTH': 6,
 ' QSO QSO_SOUTH BGS_ANY': 1,
 ' QSO QSO_SOUTH STD_WD BGS_ANY MWS_ANY': 1}

In [41]:
zcat_large[:10]

TARGETID,BRICKNAME,SPECTYPE,Z,ZERR,ZWARN,NUMOBS
int64,str8,str10,float32,float32,int32,int32
35185827130641806,,QSO,2.0,0.0,0,4
35186042583647168,,QSO,2.0,0.0,0,4
35186048573113651,,QSO,2.0,0.0,0,4
35186048761859902,,QSO,2.0,0.0,0,4
35186060317166607,,QSO,2.0,0.0,0,4
35186101530397717,,QSO,2.0,0.0,0,4
35186101748502946,,QSO,2.0,0.0,0,4
35186107578583476,,QSO,2.0,0.0,0,4
35186165778745948,,QSO,2.0,0.0,0,4


In [42]:
in_mtl_large[:10]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,PRIORITY_INIT,SUBPRIORITY,NUMOBS_INIT,HPXPIXEL,NUMOBS_MORE,PRIORITY,OBSCONDITIONS
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int32
175.358338455,2.82834728661,35185827130641806,3458764522410737668,262148,1280,3400,0.304034474116,4,26999,0,2,7
161.632699344,11.6849594655,35186042583647168,262148,0,0,3400,0.066566431465,4,27512,4,3400,1
165.761079191,11.9375467863,35186048573113651,1152921504607109124,262148,0,3400,0.986820059528,4,27509,4,3400,5
177.194196895,12.062393253,35186048761859902,262148,0,0,3400,0.768395743079,4,27872,4,3400,1
161.382869727,12.5366379114,35186060317166607,3458764531000672260,262148,2,3400,0.410806412807,4,27515,0,2,7
163.74201943,14.2736572006,35186101530397717,262148,0,0,3400,0.36198326493,4,28202,4,3400,1
177.016499723,14.3679191504,35186101748502946,262148,0,0,3400,0.294375182193,4,27881,4,3400,1
175.332046289,14.4957383573,35186107578583476,262148,0,0,3400,0.197275485253,4,27882,4,3400,1
175.389783072,17.0186553116,35186165778745948,262148,0,0,3400,0.466633580746,4,28230,4,3400,1


In [43]:
in_targets_large[:10]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,PRIORITY_INIT,SUBPRIORITY,NUMOBS_INIT,HPXPIXEL,NUMOBS_MORE,PRIORITY,OBSCONDITIONS
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int32
175.358338455,2.82834728661,35185827130641806,3458764522410737668,262148,1280,3400,0.304034474116,4,26999,4,3400,7
161.632699344,11.6849594655,35186042583647168,262148,0,0,3400,0.066566431465,4,27512,4,3400,1
165.761079191,11.9375467863,35186048573113651,1152921504607109124,262148,0,3400,0.986820059528,4,27509,4,3400,5
177.194196895,12.062393253,35186048761859902,262148,0,0,3400,0.768395743079,4,27872,4,3400,1
161.382869727,12.5366379114,35186060317166607,3458764531000672260,262148,2,3400,0.410806412807,4,27515,4,3400,7
163.74201943,14.2736572006,35186101530397717,262148,0,0,3400,0.36198326493,4,28202,4,3400,1
177.016499723,14.3679191504,35186101748502946,262148,0,0,3400,0.294375182193,4,27881,4,3400,1
175.332046289,14.4957383573,35186107578583476,262148,0,0,3400,0.197275485253,4,27882,4,3400,1
175.389783072,17.0186553116,35186165778745948,262148,0,0,3400,0.466633580746,4,28230,4,3400,1


In [44]:
name_id

{35185827130641806: ' QSO QSO_SOUTH STD_FAINT BGS_ANY MWS_ANY',
 35186042583647168: ' QSO QSO_SOUTH',
 35186048573113651: ' QSO QSO_SOUTH BGS_ANY',
 35186048761859902: ' QSO QSO_SOUTH',
 35186060317166607: ' QSO QSO_SOUTH STD_WD BGS_ANY MWS_ANY',
 35186101530397717: ' QSO QSO_SOUTH',
 35186101748502946: ' QSO QSO_SOUTH',
 35186107578583476: ' QSO QSO_SOUTH',
 35186165778745948: ' QSO QSO_SOUTH'}

In [45]:
def consolidate_fassigned(fba_path):
    fba_files = glob.glob(os.path.join(fba_path, 'tile-*fits'))

    # getting all the targetids of the assigned fibers
    print('reading individual fiberassign files')
    favail = list()
    for i_tile, tile_file in enumerate(fba_files):
        if i_tile%50 ==0:
            print(i_tile)
        id_favail, header = fits.getdata(tile_file, 'FIBERASSIGN', header=True)
        favail.extend(id_favail['TARGETID'])
    return np.array(favail)

In [46]:
mtl_names = {0:"strategy_A/targets/gray_subset_dr8_mtl_dark_gray_northern_cap.fits",
             1:"strategy_A/targets/dark0_subset_dr8_mtl_dark_gray_northern_cap.fits",
             2:"strategy_A/targets/dark1_subset_dr8_mtl_dark_gray_northern_cap.fits",
             3:"strategy_A/targets/dark2_dark3_subset_dr8_mtl_dark_gray_northern_cap.fits"}
zcat_names = {0:"strategy_A/zcat/gray_zcat.fits",
             1:"strategy_A/zcat/dark0_zcat.fits",
             2:"strategy_A/zcat/dark1_zcat.fits",
             3:"strategy_A/zcat/dark2_dark3_zcat.fits"}
tile_names = {0:'strategy_A/fiberassign_gray/', 
             1:'strategy_A/fiberassign_dark0/',
             2:'strategy_A/fiberassign_dark1/',
             3:'strategy_A/fiberassign_dark2_dark3/'}

mtl_data = {}
zcat_data = {}
tile_data = {}
tile_files = {}
for k in mtl_names.keys():
    mtl_data[k] = Table.read(mtl_names[k])
    zcat_data[k] = Table.read(zcat_names[k])
    tile_data[k] = consolidate_fassigned(tile_names[k])
    tile_files[k] = glob.glob(os.path.join(tile_names[k], 'tile-*fits'))


reading individual fiberassign files
0
reading individual fiberassign files
0
reading individual fiberassign files
0
50
reading individual fiberassign files
0
50


In [47]:
def print_stats(id_to_track):
    print("ID_TO_TRACK", id_to_track)
    for k in range(4):
        ii = mtl_data[k]['TARGETID']==id_to_track
        jj = zcat_data[k]['TARGETID']==id_to_track
        kk = np.count_nonzero(tile_data[k]==id_to_track)
        fa = Table([[kk]], names=(['NFIBER']))
        if(np.count_nonzero(jj)):
            a = hstack((mtl_data[k][['PRIORITY', 'NUMOBS_MORE']][ii], fa, zcat_data[k][['NUMOBS']][jj]))
        else:
            a = hstack((mtl_data[k][['PRIORITY', 'NUMOBS_MORE']][ii], fa))      
        print(a)
    print()

In [48]:
#ids_to_track = [35185796730325505, 35186182941835863, 35186188885166096, 35186188750949832, 35186194501340726, 35186107519861231]
ids_to_track = [35185827130641806, 35185893358698744, 35186054600327316 , 35186107452755511, 35186165573224800]
ids_to_track = [35185790698917045, 35185833006861227, 35185833027834038, 35185821090841787]
ids_to_track = [35185796747101770, 35185796772268362, 35185796885515329, 35185796910679801]
ids_to_track = [35185790673748964,35185790682139455, 35185790698917655, 35185790745053758, 35185814883271540]
ids_to_track = [35185808809922194, 35185808826699688, 35185808856058923, 35185808872836871]
ids_to_track = list(name_id.keys())

for id_to_track in ids_to_track:
    print_stats(id_to_track)

ID_TO_TRACK 35185827130641806
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3400           4      1      1
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
       2           0      1      2
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
       2           0      1      3


PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
       2           0      1      4

ID_TO_TRACK 35186042583647168
PRIORITY NUMOBS_MORE NFIBER
-------- ----------- ------
    3400           4      0
PRIORITY NUMOBS_MORE NFIBER
-------- ----------- ------
    3400           4      0
PRIORITY NUMOBS_MORE NFIBER
-------- ----------- ------
    3400           4      0
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3400           4      4      4

ID_TO_TRACK 35186048573113651
PRIORITY NUMOBS_MORE NFIBER
-------- ----------- ------
    3400           4      0
PRIORITY NUMOBS_MORE NFIBER
-------- ----------- ------
    3400           4      0
PRIORITY NUMOBS_MORE NFIBER
-------- ----------- ------
    3400           4      0
PRIORITY NUMOBS_MORE NFIBER NUMOBS
-------- ----------- ------ ------
    3400           4      4      4

ID_TO_TRACK 35186048761859902
PRIORITY NUMOBS_MORE NFIBER
-------- ----------- ------
    3400           4      0
PRIO

In [99]:
myquickcat = reload(myquickcat)
new_zcat = myquickcat.quickcat(tile_files[3], mtl_data[3], truth, zcat=zcat_data[2], perfect=True)name_id

INFO:myquickcat.py:585:quickcat: Sat Sep 14 13:01:07 2019 QC Reading 98 tiles
INFO:myquickcat.py:608:quickcat: Sat Sep 14 13:01:09 2019 QC Read 98 tiles
INFO:myquickcat.py:630:quickcat: Sat Sep 14 13:01:09 2019 QC Trimming truth to just observed targets
INFO:myquickcat.py:637:quickcat: Sat Sep 14 13:01:10 2019 QC Constructing new redshift catalog
INFO:myquickcat.py:649:quickcat: Sat Sep 14 13:01:10 2019 QC Adding ZERR and ZWARN
INFO:myquickcat.py:667:quickcat: Sat Sep 14 13:01:10 2019 QC Adding NUMOBS column
nobs 1
1. newzcat NUMOBS NUMOBS
------
     1
1. zcat NUMOBS NUMOBS
------
     3
INFO:myquickcat.py:680:quickcat: Sat Sep 14 13:01:12 2019 QC Merging previous zcat
2. newzcat NUMOBS NUMOBS
------
     1
2. zcat NUMOBS NUMOBS
------
     3
3. newzcat NUMOBS NUMOBS
------
     4
3. zcat NUMOBS NUMOBS
------
     4
newzcat NUMOBS NUMOBS
------
     4
INFO:myquickcat.py:738:quickcat: Sat Sep 14 13:01:14 2019 QC done


In [57]:
id_to_track = ids_to_track[-4]
pass_id = 3
print('ID_TO_TRACK', id_to_track)

ll = zcat_data[pass_id-1]['TARGETID']==id_to_track
ii = mtl_data[pass_id]['TARGETID']==id_to_track
jj = np.count_nonzero(tile_data[pass_id]==id_to_track)
kk= new_zcat['TARGETID']==id_to_track
fa = Table([[jj]], names=(['NFIBER']))

print(zcat_data[pass_id-1]['NUMOBS'][ll])
print(mtl_data[pass_id]['NUMOBS_MORE'][ii])
print(fa)
print(new_zcat['NUMOBS'][kk])



ID_TO_TRACK 35186188885166096
NUMOBS
------
     3
NUMOBS_MORE
-----------
          1
NFIBER
------
     1
NUMOBS
------
     5
